In [1]:
import typing
import os

import pandas
import transformers

import cltrier_lib

import twon_agents

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
MODELS: typing.Dict[str, str] = {
    "base": "meta-llama/Llama-3.2-3B-Instruct", 
    "adapter": "simon-muenker/TWON-Agent-OSN-Posts-de"
}

In [4]:
INSTRUCTION = cltrier_lib.inference.schemas.Message(role="system", content="You are a Alice Weidel member of the German political party AFD. Post a Tweet about the following topic:")

In [5]:
SAMPLES: typing.List[str] = [
    "FDP, Kernkraftwerke, Ostwestfalen-Lippe",
    "Energie, Umwelt, Sicherheit",
    "Autofreie Stadt, Umweltverbesserung, Stadtkonzept",
    "Sozialtourismus, Flüchtlinge, Führungsanspruch",
    "Tempolimit",
]

In [6]:
pipelines: typing.Dict[str, transformers.Pipeline] = twon_agents.util.load_pipelines(MODELS)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [10]:
responses = []

for idx, post in enumerate(SAMPLES):
    for model, pipeline in pipelines.items():
        chat = cltrier_lib.inference.schemas.Chat(messages=[
            INSTRUCTION,
            cltrier_lib.inference.schemas.Message(role="user", content=post)
        ])

        reply = pipeline(
            pipeline.tokenizer.apply_chat_template(chat, tokenize=False), 
            max_new_tokens=128,
            return_full_text=False
        )[0]["generated_text"].split("\n\n")[1]

        responses.append(
            dict(
                id=idx,
                model=model,
                post=post,
                reply=reply
            )
        )
        

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [11]:
pandas.set_option('display.max_colwidth', None)

(
    pandas.DataFrame(responses)
    .set_index(["id", "model"])
    .style.set_properties(**{'text-align': 'left'})
)